# Journals

This notebook is used for processing daily journals.

In [1]:
import pandas as pd
import re

# These will be specific to the journal data file used
JOURNAL_FILENAME = 'journals.txt'
HEADER_REGEX = r'(\d+\/\d+\/\d+)'
GRATEFUL_REGEX = r'\-(.*)'

Lets put the journals into two seperate dataframes. One will include the bulleted grateful items for that day. The other will include the raw paragraphs. 

In [2]:
raw_paragraphs = []
raw_gratefuls = []

date = 'Empty'
grateful = 'Empty'
paragraph = 'Empty'

# Import journal data from text file
with open(JOURNAL_FILENAME) as file:
    for line in file:
        if 'Journal' in line:
            date = re.search(HEADER_REGEX, line).group(1)
        elif '-' in line:
            grateful = re.search(GRATEFUL_REGEX, line).group(1)
            raw_gratefuls.append((date, grateful))
        elif line not in ['\n', '#journal\n']:
            raw_paragraphs.append((date, line))
        else:
            pass
                
# Convert to dataframes
grateful = pd.DataFrame(raw_gratefuls, columns=['Date', 'Grateful'])
paragraph = pd.DataFrame(raw_paragraphs, columns=['Date', 'Line'])

In [6]:
grateful.head()
paragraph.head()

,Date,Grateful
0,26/03/2018,inforces the idea to myself that Youtube is an...
1,26/03/2018,Read paper TCN. Excited about work I will do ...
2,25/03/2018,constructive internet. I need to get back to m...
3,25/03/2018,Chinese food with Jenna
4,25/03/2018,Finally won fortnite
